Going through some introductions into langchain

Quick intro into usage

In [5]:
import os
# Check to make sure openai api key is there
# I place the OPENAI_API_KEY in the .env file in top level of notebook
# os.getenv("OPENAI_API_KEY")

# If not set you can manually set it here 
# from langchain.llms import OpenAI
# llm = OpenAI(openai_api_key="...")

Easy call just like chatgpt

In [6]:
from langchain.chat_models import ChatOpenAI

chat_model = ChatOpenAI(model="gpt-3.5-turbo")

In [7]:
chat_model.predict("what is katy perry's age")

'As of September 2021, Katy Perry is 36 years old. She was born on October 25, 1984.'

shows limitations of current event knowledge

Lets structure this call to show the power of using the api versus chatgpt

In [8]:
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.chains import LLMChain

def get_celebrity_age(celebrity: str) -> str:
    """Determines celebrity age"""
    chat_prompt = ChatPromptTemplate.from_messages(
        messages=[
            SystemMessagePromptTemplate.from_template(
                "Your input is a celebrity name."
                "Output the age of the celebrity."
            ),
            HumanMessagePromptTemplate.from_template("{input}"),
        ]
    )
    chain = LLMChain(llm=ChatOpenAI(model="gpt-3.5-turbo"), prompt=chat_prompt, verbose=False)
    return chain.run(celebrity)

In [9]:
get_celebrity_age("Katy Perry")

'Katy Perry is currently 36 years old.'

Lets fix this for current event 

We will be using serpapi to serf the web to get current events

please make a free acct on serpapi.com and add SERPAPI_API_KEY to .env file

In [11]:
from langchain.utilities import SerpAPIWrapper

search = SerpAPIWrapper()

search.run("How old is Katy perry?")

'39 years'

IT IS DIFFERENT

Lets chain serpapi with langchain

In [12]:
from langchain.agents import load_tools, initialize_agent, AgentType

tools = load_tools(["serpapi"], llm=ChatOpenAI(model="gpt-3.5-turbo"))
agent = initialize_agent(tools, ChatOpenAI(model="gpt-3.5-turbo"), agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

first_response = agent.run("How old is Katy perry?")



> Entering new AgentExecutor chain...
I should search for Katy Perry's age.
Action: Search
Action Input: "Katy Perry age"
Observation: 39 years
Thought:I now know the final answer
Final Answer: Katy Perry is 39 years old.

> Finished chain.


Prompt tune to get structured data

In [13]:
response = agent.run("How old is Katy perry? output as json formatted dict with name of celebrity and age as fields")



> Entering new AgentExecutor chain...
I should search for Katy Perry's age and find a way to output it as a JSON formatted dict.
Action: Search
Action Input: "Katy Perry age"
Observation: 39 years
Thought:I can now create a JSON formatted dict with the name of the celebrity (Katy Perry) and her age (39).
Final Answer: {"name": "Katy Perry", "age": 39}

> Finished chain.


In [14]:
import json
dict_example = json.loads(response)
print(dict_example)

{'name': 'Katy Perry', 'age': 39}


Forcing structure with tagging

In [15]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import create_tagging_chain

In [16]:
# Schema
schema = {
    "properties": {
        "celebrity_name": {"type": "string"},
        "age": {"type": "integer"},
    }
}

chain = create_tagging_chain(schema, llm=ChatOpenAI(model="gpt-3.5-turbo"))

In [17]:
# reusing past response
print(first_response)
chain.run(first_response)

Katy Perry is 39 years old.


{'celebrity_name': 'Katy Perry', 'age': 39}

In [18]:
extra_response = "Justin Au-Yeung is 27 years old"
chain.run(extra_response)

{'celebrity_name': 'Justin Au-Yeung', 'age': 27}